# Data Exploration in Python (without pandas)

In this notebook, we will use python to parse some datasets.

### Amazon Reviews Dataset

Up until now, we've worked mainly with CSV data that has been easy to import. However, often its not the modeling part that is difficult (once we understand the models) but getting data into a format that we can use for modeling.

Today we'll look at 2 datasets from Amazon. The first is a collection of movie reviews (the original set of 7 million reviews is also available) and the second is meta data on products including which customers reviewed it. However, neither is in a ready-to-use format so we must work on that first.

### Amazon Movie Reviews

- We use limited version of this data, which is stored in our repo under `/data/amazon/small-movies.txt`
- The full data set is available
<a href="http://snap.stanford.edu/data/web-Movies.html" target="_blank">here</a>.

In [1]:
# Replace the following path to your own local path
path_to_repo = "/Users/ruben/repo/personal/ga/DAT-23-NYC/"
path_to_data = path_to_repo + "data/amazon/"

In [2]:
# Load the dataset into a list of separate lines in python
with open(path_to_data + 'small-movies.txt') as f:
    lines = [line for line in f]

#### Exercise 1

Create a tab-separated file from the file above the contains the following columns: productId, userId, review text, helpfulness score (as a numeric value) and review score (as a numeric value).
Note: What are the issues with helpfulness? How can you resolve them?

In [48]:
reviews = []
record = {}

for line in lines:
    if line.startswith('product/productId: '):
        if record:
            reviews.append(record)
        record = {'productId': line[len('product/productId: '):].strip()}
    elif line.startswith('review/userId: '):
        record['userId'] = line[len('review/userId: '):].strip()
    elif line.startswith('review/helpfulness: '):
        n, d = line[len('review/helpfulness: '):].strip().split('/')
        try:
            record['helpfulness'] = float(n) / float(d)
        except:
            record['helpfulness'] = None
    elif line.startswith('review/score: '):
        record['score'] = float(line[len('review/score: '):].strip())
    elif line.startswith('review/text: '):
        record['text'] = line[len('review/text: '):].strip()

if record:
    reviews.append(record)

In [49]:
# Create list of rows to be written to file
rows = []

columns = ['productId', 'userId', 'score', 'helpfulness', 'text']
rows.append("\t".join(columns))  # first write header row
for review in reviews:
    rows.append("\t".join([str(review[col]) for col in columns]))

In [53]:
rows[0:2]

['productId\tuserId\tscore\thelpfulness\ttext',
 'B003AI2VGA\tA141HP4LYPWMSR\t3.0\t1.0\tSynopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the area.<br /><br />Critique: Pos

In [54]:
# Write rows to disk
output_file = path_to_data + 'small-movies-results.csv'
with open(output_file, 'w') as f:
    for row in rows:
        f.write(row + '\n')

Let's ensure that you have a properly formatted TSV and that you can parse it back in with Pandas.

In [58]:
# Quick check if it indeed works
import pandas as pd  # more on pandas later
df = pd.read_csv(output_file, sep='\t', header=0)
df.head()

,productId,userId,score,helpfulness,text
0,B003AI2VGA,A141HP4LYPWMSR,3,1.0,"Synopsis: On the daily trek from Juarez, Mexic..."
1,B003AI2VGA,A328S9RN3U5M68,3,1.0,THE VIRGIN OF JUAREZ is based on true events s...
2,B003AI2VGA,A1I7QGUDP043DG,5,0.8,The scenes in this film can be very disquietin...
3,B003AI2VGA,A1M5405JH9THP9,3,1.0,THE VIRGIN OF JUAREZ (2006)<br />directed by K...
4,B003AI2VGA,ATXL536YX71TR,3,1.0,"Informationally, this SHOWTIME original is ess..."


In [59]:
# How many rows do we have?
len(df)

1111

### Amazon Metadata

A limited version of this data is available here and the full data set is available here

- We use limited version of this data, which is stored in our repo under `/data/amazon/small-amazon-meta.txt`
- The full data set is available
<a href="http://snap.stanford.edu/data/amazon-meta.html" target="_blank">here</a>.

The task here is to parse this file into a collection of product ids (ASIN), title and list of customers (by id) who have reviewed the product.

```
productID, title, [customer1, customer2]
```

In [22]:
# Load the dataset into a list of separate lines in python
with open(path_to_data + 'small-amazon-meta.txt') as f:
    lines = [line for line in f]

In [54]:
products = []
metadata = {}

line_no = 0
while line_no < len(lines):
    line = lines[line_no]
    if line.startswith('ASIN:'):
        if metadata:
            products.append(metadata)
        metadata = {'productID': line[5:].strip()}
    elif line.startswith('  title:'):
        metadata['title'] = line[len('  title:'):].strip()
    elif line.startswith('  reviews:'):
        number_of_customers = int(line[len('  reviews: total:'):].split()[0])
        metadata['customers'] = []
        for i in xrange(number_of_customers):
            line_no += 1
            line = lines[line_no]
            pos = line.find('cutomer:')  # NOTE the typo!
            if pos > -1:
#                 print line
                metadata['customers'].append(line[pos + len('cutomer:'):].split()[0])
    line_no += 1

In [78]:
for product in products[:5]:  # just print first 5
    print product['productID'] + ', ' + product.get('title', '') + ',',
    print '[' + ', '.join([customer for customer in product.get('customers', [])]) + ']'

0771044445, , []
0827229534, Patterns of Preaching: A Sermon Sampler, [A2JW67OY8U6HHK, A2VE83MZF98ITY]
0738700797, Candlemas: Feast of Flames, [A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM, A1BDAI6VEYMAZA, A2P6KAWXJ16234, AMACWC3M7PQFR, A3GO7UV9XX14D8, A1GIL64QK68WKL, AEOBOF2ONQJWV, A3IGHTES8ME05L, A1CP26N8RHYVVO, ANEIANH0WAT9D]
0486287785, World War II Allied Fighter Planes Trading Cards, [A3IDGASRQAW8B2]
0842328327, Life Application Bible Commentary: 1 and 2 Timothy and Titus, [A2591BUPXCS705]


In [79]:
# Let's write it to file, and try to load back. I am using | to 
with open(path_to_data + 'small-amazon-meta-results.csv', 'w') as f:
    for product in products:
        f.write("%s|%s|%s\n" % (product['productID'],
                              product.get('title'), 
                              [customer for customer in product.get('customers', [])]))

In [80]:
pd.read_csv(path_to_data + 'small-amazon-meta-results.csv', header=None, sep='|').head()

,0,1,2
0,0771044445,None,[]
1,0827229534,Patterns of Preaching: A Sermon Sampler,"['A2JW67OY8U6HHK', 'A2VE83MZF98ITY']"
2,0738700797,Candlemas: Feast of Flames,"['A11NCO6YTE4BTJ', 'A9CQ3PLRNIR83', 'A13SG9ACZ..."
3,0486287785,World War II Allied Fighter Planes Trading Cards,['A3IDGASRQAW8B2']
4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,['A2591BUPXCS705']


In [69]:
# NOTE: while debugging, you might want to use try/except
for i in xrange(100):
    try:
        print 10/(50-i),
    except:
        print "Error!"
        print "at i =", i

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 2 3 5 10 Error!
at i = 50
-10 -5 -4 -3 -2 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1


#### Exercise 2

- (**) If you'd like, create a review class that holds the customer id and star rating. Use this to output a product id, title and list of reviews.
- (*) Create a product class that holds the id, title and collection of reviews.

In [81]:
## Okay, this one is a little too much for the experts.
## Contact me if you're interested.

### Lahman Baseball Dataset

Available here: 
<a href="http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip">Lahman Baseball Dataset</a>.

- Without using Pandas, read in Salaries.csv and output average salary by playerID.

In [87]:
salaries = {}
with open('/Users/ruben/Downloads/lahman-csv_2014-02-14/Salaries.csv') as f:
    f.readline()  # read first line of headers: yearID,teamID,lgID,playerID,salary
    for line in f:
        yearID, teamID, lgID, playerID, salary = line.split(',')
        if playerID not in salaries:
            salaries[playerID] = []
        salaries[playerID].append(int(salary.strip()))

In [90]:
avg_salaries = {player: sum(salaries[player]) / float(len(salaries[player])) 
                for player in salaries}

In [92]:
# avg_salaries

- (**) Without using Pandas, read in Salaries.csv and Master.csv and output salary by nameFirst and nameLast.

In [97]:
# Look up which columns we need in Master
header = 'playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID'
print list(enumerate(header.split(',')))

[(0, 'playerID'), (1, 'birthYear'), (2, 'birthMonth'), (3, 'birthDay'), (4, 'birthCountry'), (5, 'birthState'), (6, 'birthCity'), (7, 'deathYear'), (8, 'deathMonth'), (9, 'deathDay'), (10, 'deathCountry'), (11, 'deathState'), (12, 'deathCity'), (13, 'nameFirst'), (14, 'nameLast'), (15, 'nameGiven'), (16, 'weight'), (17, 'height'), (18, 'bats'), (19, 'throws'), (20, 'debut'), (21, 'finalGame'), (22, 'retroID'), (23, 'bbrefID')]


In [113]:
avg_salaries_by_name = {}
with open('/Users/ruben/Downloads/lahman-csv_2014-02-14/Master.csv') as f:
    f.readline()  # read first line of headers: yearID,teamID,lgID,playerID,salary
    for line in f:
        cols = line.split(',')
        playerID, nameFirst, nameLast = cols[0], cols[13], cols[14]
        if playerID in salaries:
            avg_salaries_by_name[nameFirst + ' ' + nameLast] = avg_salaries[playerID]

In [115]:
# avg_salaries_by_name